## **Computing Binomial Moments using Eq.(8)**

**Stochastic Kinetics of mRNA Molecules in a General Transcription Model**

*Yuntao Lu and Yunxin Zhang*

School of Mathematical Sciences, Fudan University, Shanghai 200433, China

Email: `yuntaolu22@m.fudan.edu.cn` and `xyz@fudan.edu.cn`

This script is written to time computations using Eq.(8), generating date used to plot Figure 4. 

In [4]:
import scipy.linalg as linalg
import numpy as np
import time
import math

In [12]:
def moment(D0, D1, M=200, threshold=1e-32):
    """
    Compute the first M+1 binomial moments (starting from B_0=1) of mRNA copy number 
    with given parameters in the transcription model (1) in our manucript.
  
    Input:
    D0 (np.ndarray): Transition rate matrix for non-transcription events
    D1 (np.ndarray): Transition rate matrix for transcription events
    M (int): Number of binomial moments to compute (returns M+1 moments including B_0)
    threshold (float, optional): Tolerance used for determining early stop. If the value of
                                 binomial moment is below this threshold, the computation is terminated.
                                 Default is 1e-32.
    
    Output:
    list: Binomial moments [B_0, B_1, ..., B_M] where B0=1 by definition
    
    Raises:
    TypeError: If input types are incorrect
    ValueError: If matrices are invalid or dimensions mismatch
    RuntimeError: For numerical computation errors
    """
    # ==============================================
    # INPUT VALIDATION AND SAFETY CHECKS
    # ==============================================
    
    # Type checking for input parameters
    if not isinstance(D0, np.ndarray) or not isinstance(D1, np.ndarray):
        raise TypeError("Input matrices D0 and D1 must be NumPy arrays")
    if not isinstance(M, int):
        raise TypeError("M must be an integer")
    
    # Matrix dimension validation
    if D0.ndim != 2 or D1.ndim != 2:
        raise ValueError("Input matrices must be 2-dimensional arrays")
    
    n0, m0 = D0.shape
    n1, m1 = D1.shape
    
    # Check if matrices are square
    if n0 != m0:
        raise ValueError("D0 must be a square matrix")
    if n1 != m1:
        raise ValueError("D1 must be a square matrix")
    
    # Check matrix dimension consistency
    if n0 != n1:
        raise ValueError("D0 and D1 must have the same dimension")
    n = n0  # Dimension of the system
    
    # Validate moment count
    if M < 0:
        raise ValueError("M must be a non-negative integer")
    
    # Hydrolysis rate (normalized to 1)
    d = 1.0  

    
    # Construct Q-matrix D
    D = D0 + D1
    
    # ==============================================
    # Q-MATRIX VALIDATION
    # ==============================================
    tol = 1e-8  # Numerical tolerance for checks
    
    # Validate Q-matrix properties:
    # 1. Row sums should be zero (within tolerance)
    row_sums = np.sum(D, axis=1)
    if not np.allclose(row_sums, 0, atol=tol):
        max_error = np.max(np.abs(row_sums))
        raise ValueError(f"Invalid generator matrix: Row sums should be zero "
                         f"(max error: {max_error:.2e})")
    
    # 2. Off-diagonal elements should be non-negative
    D_offdiag = D.copy()
    np.fill_diagonal(D_offdiag, 0)
    if np.any(D_offdiag < -tol):
        min_value = np.min(D_offdiag)
        raise ValueError(f"Invalid generator matrix: Negative off-diagonals "
                         f"(min value: {min_value:.2e})")
    
    # 3. Diagonal elements should be non-positive
    diag_elements = np.diag(D)
    if np.any(diag_elements > tol):
        max_value = np.max(diag_elements)
        raise ValueError(f"Invalid generator matrix: Positive diagonals "
                         f"(max value: {max_value:.2e})")
    
    # Check numerical stability
    # inf_norm_D1 = linalg.norm(D1, np.inf)
    # if inf_norm_D1 > 9:
        # print(f"Warning: High matrix norm ||D1||_∞ = {inf_norm_D1:.2f} - "
               # "potential numerical instability")
    
    # ==============================================
    # COMPUTATION of Invariant Distribution of D
    # ==============================================
    try:
        # Construct linear system to calculate stationary distribution of the underlying Markov chain characterized by D:
        # D^T π = 0 with constraint sum(π) = 1
        DT = D.T.copy()
        
        # Replace first row with constraint equation
        DT[0, :] = np.ones(n)
        
        # Right-hand side vector: [1, 0, 0, ...]
        b = np.zeros(n)
        b[0] = 1.0
        
        # Check matrix condition number
        cond_num = np.linalg.cond(DT)
        if cond_num > 1e12:
            print(f"Warning: Ill-conditioned matrix (cond={cond_num:.2e})")
        
        # Solve for stationary distribution pi (pi is a 2D NumPy array of shape (1, n))
        pi = np.linalg.solve(DT, b).reshape(1, n)
    
    except np.linalg.LinAlgError as e:
        print(f"Linear system solver failed: {str(e)}.")
        print(f'Make sure that the Q-matrix D is irreducible.')
        raise
    
    # Vector of ones (e is a 2D NumPy array of shape (n, 1))
    e = np.ones((n, 1))
    
    # ==============================================
    # BINOMIAL MOMENTS COMPUTATION
    # ==============================================
    
    
    binomial_moments = [1.0]  # strat with B_0 = 1
    
    # Early return for trivial case
    if M == 0:
        return binomial_moments
    
    # ==============================================
    # Start timing the computation
    start_time = time.time()
    # ==============================================
    
    # Compute first moment B1
    B1 = (pi @ D1 @ e) / d
    binomial_moments.append(float(B1[0, 0]))
    
    
    # Compute higher-order moments (B2 to BM)
    if M > 1:
        B_vec = pi @ D1  # Initialize moment vector
    
        index_at_threshold = None
        
        for i in range(2, M + 1):
            # Construct matrix for current iteration
            A = (i - 1) * d * np.eye(n) - D
            
            # Compute the inverse of (i-1)*d*I - D using scipy.linalg.inv()    
            invA = linalg.inv(A)
    
            # Compute next vector: B_vec = B_vec * inv(A) * D1
            B_vec = B_vec @ invA @ D1
            moment_i = (B_vec @ e) / (i * d)
            moment_val = float(moment_i[0, 0])
            
            # Store computed moment
            binomial_moments.append(moment_val)
            
            # Early termination if moments become negligible
            # if moment_val < threshold:
            #     if index_at_threshold is None:
            #         index_at_threshold = i
            #         print(f"Moments below threshold at i={i}")
            #         break
    
    # ==============================================
    # FINAL VALIDATION AND OUTPUT
    # ==============================================
    # Verify we computed the correct number of moments
    # if len(binomial_moments) != M + 1:
    #     print(f"Expected {M+1} moments, got {len(binomial_moments)}. ")
    #     print(f'\nEarly stop due to threshold={threshold}')
    
    # Print performance statistics
    total_time = time.time() - start_time
    print(f"Computation for {n0}-th order model is completed in {total_time:.4f} seconds")
    # print(f"\nComputed {len(binomial_moments)} binomial moments")
    
    return total_time,binomial_moments

In [11]:
# import parameter matrices prepared for figures in the paper
import Parameters_for_Figures

In [14]:
times=[]
for i in range(1,201):
    time1=moment(Parameters_for_Figures.generate_D0n(i),
                     Parameters_for_Figures.generate_D1n(i),
                 200)[0]
    time2=moment(Parameters_for_Figures.generate_D0n(i),
                     Parameters_for_Figures.generate_D1n(i),
                 200)[0]
    time3=moment(Parameters_for_Figures.generate_D0n(i),
                     Parameters_for_Figures.generate_D1n(i),
                 200)[0]
    times.append(float(np.average([time1,time2,time3])))

Computation for 1-th order model is completed in 0.0243 seconds
Computation for 1-th order model is completed in 0.0151 seconds
Computation for 1-th order model is completed in 0.0130 seconds
Computation for 2-th order model is completed in 0.0125 seconds
Computation for 2-th order model is completed in 0.0109 seconds
Computation for 2-th order model is completed in 0.0093 seconds
Computation for 3-th order model is completed in 0.0082 seconds
Computation for 3-th order model is completed in 0.0074 seconds
Computation for 3-th order model is completed in 0.0070 seconds
Computation for 4-th order model is completed in 0.0064 seconds
Computation for 4-th order model is completed in 0.0063 seconds
Computation for 4-th order model is completed in 0.0060 seconds
Computation for 5-th order model is completed in 0.0060 seconds
Computation for 5-th order model is completed in 0.0056 seconds
Computation for 5-th order model is completed in 0.0056 seconds
Computation for 6-th order model is comp

/tmp/ipykernel_5128/1699435038.py:162: RuntimeWarning: overflow encountered in matmul
  moment_i = (B_vec @ e) / (i * d)


Computation for 108-th order model is completed in 0.1687 seconds
Computation for 108-th order model is completed in 0.1351 seconds


/tmp/ipykernel_5128/1699435038.py:161: RuntimeWarning: overflow encountered in matmul
  B_vec = B_vec @ invA @ D1


Computation for 109-th order model is completed in 0.1907 seconds
Computation for 109-th order model is completed in 0.1809 seconds
Computation for 109-th order model is completed in 0.1831 seconds
Computation for 110-th order model is completed in 0.1354 seconds
Computation for 110-th order model is completed in 0.1922 seconds
Computation for 110-th order model is completed in 0.1674 seconds
Computation for 111-th order model is completed in 0.1414 seconds
Computation for 111-th order model is completed in 0.1886 seconds
Computation for 111-th order model is completed in 0.1830 seconds
Computation for 112-th order model is completed in 0.1868 seconds
Computation for 112-th order model is completed in 0.1850 seconds
Computation for 112-th order model is completed in 0.1417 seconds
Computation for 113-th order model is completed in 0.1892 seconds
Computation for 113-th order model is completed in 0.1771 seconds
Computation for 113-th order model is completed in 0.1895 seconds
Computatio

In [15]:
# Save the results in a `.npy` file
# np.save('Main_times.npy', times)

In [16]:
# load .npy file
loaded_array = np.load('Main_times.npy')
loaded_list_from_array = loaded_array.tolist()
print(loaded_list_from_array)

[0.017479340235392254, 0.010891199111938477, 0.007544994354248047, 0.006255388259887695, 0.005748589833577474, 0.0054627259572347, 0.005471229553222656, 0.005526622136433919, 0.00612489382425944, 0.0066649119059244795, 0.006467342376708984, 0.006778319676717122, 0.00565338134765625, 0.005725542704264323, 0.005891879399617513, 0.005863030751546224, 0.006070693333943685, 0.006250063578287761, 0.006417671839396159, 0.006469249725341797, 0.006787220637003581, 0.006890535354614258, 0.007974942525227865, 0.007248481114705403, 0.007611274719238281, 0.007734060287475586, 0.008041620254516602, 0.008054574330647787, 0.008403778076171875, 0.008725722630818685, 0.00958720842997233, 0.008910338083902994, 0.009437640508015951, 0.009612798690795898, 0.010058164596557617, 0.01019446055094401, 0.010800758997599283, 0.011470953623453775, 0.011380354563395182, 0.011371771494547525, 0.012075980504353842, 0.012183745702107748, 0.013129234313964844, 0.012829860051472982, 0.013636191685994467, 0.013677755991